# **PROYECTO FINAL | BIG DATA & ANALYTICS | MUNDIAL QATAR 2022**


<img src=https://sc2.elpais.com.uy/files/article_main_up/uploads/2019/09/03/5d6eb181e460e.jpeg>

## **Extract & Load Pipeline (Web Scraping | Google Cloud Storage | Big Query)** 

El presente programa, se encaga de extraer información de distintas páginas web, utilizando web scraping, generar archivos .CSV, almacenar los archivos en Gooogle Cloud Storage y cargar los datos en Bigquery.

### ***Importar librerías***

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import ssl
import pandas as pd
ssl._create_default_https_context = ssl._create_unverified_context
import os
from google.cloud import storage
from google.cloud import bigquery


### ***Cliente para interactuar con la API de Google Cloud Storage***

In [ ]:
gcs_client = storage.Client()

### ***Declaración de variables***

In [ ]:
#URL
url = urllib.request.Request("https://www.fifa.com/tournaments/mens/worldcup/qatar2022/qualifiers")

#URL Header
url.add_header('accept-encoding','identity')

#Response
resp = urllib.request.urlopen(url)

#Contenido general página web
soup = BeautifulSoup(resp,"html.parser")

#Búsqueda de todas las etiquetas tipo div
divs = soup.find_all("div", {"class": "ff-show-more-container_main__vowVj"})

#Tabla BigQuery
table_id = "proyecto-final-bigdata.raw_data.dim_paises"


### ***Lista para almacenar países clasificados al mundial***

In [ ]:
result={"pais":[],"clasificado":[]}

### ***Ciclo para recorrer el contenido html, y agregar los países clasificados a la lista***

In [ ]:
for country_detail in divs:
    country_div=country_detail.find("div",{"class": "row"})
    for country in country_div:
        if country.find("div",{"class": "col-1"}).text=="Q":
            result["clasificado"].append("SI")
        else:
            result["clasificado"].append("NO")
        result["pais"].append(country.find("div",{"class": "d-flex"}).text)

### ***Crear Dataframes***

In [ ]:
df_clasificados=pd.DataFrame(result)

### ***Generar archivos .CSV | Almacenar en Google Cloud Storage***

In [ ]:
df_clasificados.to_csv('gs://proyecto_final_bigdata_bucket/raw/web_scraping/mundial_Qatar_2022/paises_clasificados.csv',index=False,header=False)



In [ ]:
def load_table_uri_csv(table_id):

    # Instanciar un objeto de tipo cliente BigQuery.
    bq_client = bigquery.Client(project="proyecto-final-bigdata")

    # TODO(developer): Set table_id to the ID of the table to create.
    table_id = "proyecto-final-bigdata.raw_data.dim_paises"

    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("pais", "STRING"),
            bigquery.SchemaField("clasificado", "STRING"),
        ],
        skip_leading_rows=1,
        # The source format defaults to CSV, so the line below is optional.
        source_format=bigquery.SourceFormat.CSV,
    )
    uri = "gs://proyecto_final_bigdata_bucket/raw/web_scraping/mundial_Qatar_2022/paises_clasificados.csv"

    load_job = bq_client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = bq_client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))
    # [END bigquery_load_table_gcs_csv]

load_table_uri_csv(table_id)

Loaded 208 rows.
